In [7]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_ollama.llms import OllamaLLM
from langchain.schema.runnable import RunnableLambda

#model = OllamaLLM(model="llama3.2:latest")
model = OllamaLLM(model="gpt-oss:20b")


In [8]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system","你是一個專業的客服代表，負責回覆客戶郵件。請用友善和專業的語調回覆。請不要回覆(公司的資訊和結尾詞),我會自已手動增加"),
    ("human","請回覆這封客戶郵件：\n\n{email_content}")
])

In [9]:
def format_email_reply(ai_answer:str) -> str:
    import datetime
    date_str = datetime.datetime.now().strftime("中華民國,西元%Y-%m-%d %H:%M")
    formatted_reply = f"""{ai_answer}

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
致理客服團隊

回覆時間:{date_str}
"""

    print(f"ai回覆的內容:{formatted_reply}")

    return {"formatted_reply":formatted_reply}

format_email = RunnableLambda(format_email_reply)


In [10]:
improve_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","你是郵件品質專家。請評估這封郵件的專業度、友善度和完整度（1-10分），並提供具體的改進建議。"),
        ("human","估並改進這封郵件:\n\n{formatted_reply}")
    ]
)

In [11]:
chain = (prompt_template 
        | model
        | StrOutputParser()
        | format_email
        | improve_prompt
        | model
        | StrOutputParser()
)

In [12]:
customer_email = """您好:

我最近購買了貴公司的產品，但是發現包裝有損壞，而且產品功能與說明書不符。
請問可以退貨或換貨嗎？我已經購買一週了。

期待您的回覆。


王小明"""

In [13]:
result = chain.invoke({
    "email_content":customer_email
})

print(result)

ai回覆的內容:您好，王小明先生／女士：

感謝您即時反映包裝損壞與產品功能不符的問題，讓我們能盡快協助您處理。  
為方便核對與安排退貨／換貨，請您提供以下資訊：

1. **訂單編號**  
2. **購買日期**（或購買紀錄截圖）  
3. **包裝損壞照片**（包括破損部位、標籤等）  
4. **產品功能異常的具體情況**（舉例說明或截圖）  

收到上述資料後，我們將：

- 依照退換貨政策為您開啟退貨／換貨單  
- 重新寄送符合說明書規格的新產品（如適用）  
- 如需退還商品費用，將在收到退回商品後 5–7 個工作天內完成退款  

您可以透過網站「我的訂單」功能或直接回覆此郵件將資訊提供給我們。  
若您需要進一步協助，也歡迎隨時致電客服專線。  

期待儘速為您解決此問題，謝謝！

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
致理客服團隊

回覆時間:中華民國,西元2025-10-18 15:23

**評估（1–10分）**  

| 角度 | 分數 | 評價說明 |
|------|------|----------|
| 專業度 | **8** | 語氣正式、用詞精確，顯示出企業的專業形象。但仍有小幅可改進的地方（如結尾用詞、重複表達）。 |
| 友善度 | **7** | 感謝與邀請進一步聯繫的語句已顯示關懷，但缺乏直接道歉以及更溫馨的表達方式。 |
| 完整度 | **8** | 已列出所需資訊、後續流程與聯絡方式，基本資訊完整；仍可補充政策連結、時間規範、緊急聯絡方式等。 |

---

## 改進建議（具體可落實點）

| 位置 | 原文 | 建議修改 | 原因 |
|------|------|-----------|------|
| **郵件主旨** | 無 | **Subject：**「關於您報告的包裝損壞與功能異常 – 需要進一步資訊」 | 主旨清晰可讓收件人一眼知道郵件目的，提升開信率。 |
| **稱呼** | 您好，王小明先生／女士： | **您好，王小明先生／女士：**（保留，但可簡化為「您好，王小明先生／女士」） | 省略多餘的「／」符號，文字更簡潔。 |
| **開場** | 感謝您即時反映包裝損壞與產品功能不符的問題，讓我們能盡快協助您處理。 | **感謝您即時回報包裝損壞及功能異常的問題，為了讓我